In [33]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, merge, Dropout, Input
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/raghav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

In [35]:
#READING TRAIN DATA

fileName = 'SICK.txt'

with open(fileName, 'r') as f:
    Rows = [row.split('\t') for row in f.readlines()[1:]]

trainRows = [row for row in Rows if row[11]=='TRAIN\n']
testRows = [row for row in Rows if row[11]=='TEST\n']

trainPremises = [extract(row[1]) for row in trainRows if row[3] in labels]
trainHypotheses = [extract(row[2]) for row in trainRows if row[3] in labels]
trainLabels = [labels[row[3]] for row in trainRows if row[3] in labels]

trainData = [trainPremises, trainHypotheses, trainLabels]

testPremises = [extract(row[1]) for row in testRows if row[3] in labels]
testHypotheses = [extract(row[2]) for row in testRows if row[3] in labels]
testLabels = [labels[row[3]] for row in testRows if row[3] in labels]

testData = [testPremises, testHypotheses, testLabels]

f.close()

In [36]:
# Hyperparameters

maxLen = 50
epochs = 30
batchSize = 256
gloveDimension = 100
regularization = 4e-6

In [37]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [38]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [39]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [40]:
# Load the GloVe file, via config
embeddingsDict = dict()
glove = open("/home/raghav/NLP/data/glove.6B.100d/vec.txt")

# Iterate through the GloVe file to store all the embeddings in a dictionary
for line in glove:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddingsDict[word] = vector_dimensions

glove.close()

In [41]:
# Iterate through the embeddings and store only those that are present in our vocabulary
embeddingsMat = zeros((vocabSize, 100))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec

In [42]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

In [43]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=gloveDimension)

In [44]:
# Apply batch normalization to the two input embeddings separately

premInput = rnn(premInput)
hypoInput = rnn(hypoInput)
premInput = tf.keras.layers.BatchNormalization()(premInput)
hypoInput = tf.keras.layers.BatchNormalization()(hypoInput)

In [45]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
joint = keras.layers.concatenate([premInput, hypoInput])
joint = Dropout(0.2)(joint)
for i in range(3):
    joint = Dense(200, activation='tanh', kernel_regularizer=L2(regularization))(joint)
    joint = Dropout(0.2)(joint)
    joint = tf.keras.layers.BatchNormalization()(joint)

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function
pred = Dense(3, activation='softmax')(joint)

In [46]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
# Fitting the model using the train data

model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs)

Epoch 1/20
18/18 [==============================] - 3s 21ms/step - loss: 1.2817 - accuracy: 0.4066
Epoch 2/20
18/18 [==============================] - 0s 23ms/step - loss: 1.0858 - accuracy: 0.4661
Epoch 3/20
18/18 [==============================] - 0s 20ms/step - loss: 1.0120 - accuracy: 0.5100
Epoch 4/20
18/18 [==============================] - 0s 21ms/step - loss: 0.9685 - accuracy: 0.5303
Epoch 5/20
18/18 [==============================] - 0s 20ms/step - loss: 0.9517 - accuracy: 0.5384
Epoch 6/20
18/18 [==============================] - 0s 21ms/step - loss: 0.9148 - accuracy: 0.5661
Epoch 7/20
18/18 [==============================] - 0s 22ms/step - loss: 0.8921 - accuracy: 0.5722
Epoch 8/20
18/18 [==============================] - 0s 21ms/step - loss: 0.8836 - accuracy: 0.5794
Epoch 9/20
18/18 [==============================] - 0s 22ms/step - loss: 0.8692 - accuracy: 0.5821
Epoch 10/20
18/18 [==============================] - 0s 21ms/step - loss: 0.8668 - accuracy: 0.5765
Epoch 11/

In [48]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

20/20 [==============================] - 1s 12ms/step - loss: 0.8246 - accuracy: 0.6097
Loss =  0.8246395587921143
Acc =  0.6096616387367249
